In [2]:
import pymysql
import pandas as pd
import numpy as np
import konlpy

In [3]:
#데이터 가져오기
db = pymysql.connect(host='220.86.191.27', port=3306, user='playdata', password='playdata123!', db='bookdata', charset='utf8')

curs = db.cursor(pymysql.cursors.DictCursor)

query = '''
select *
    from books_data'''
df_books_data = pd.read_sql(query, db)

In [70]:
df_books_data.head()

,id,title,author,pub,date,price,review,rating,story,img,genre
0,1,요리코를 위해,노리즈키 린타로,모모,2020.03,13500,43,9.2,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,2,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,2021.02,14220,13,9.4,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,3,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),2020.11,16200,54,9.1,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...,http://image.yes24.com/goods/95712763/L,추리/미스터리
3,4,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,2017.02,13500,217,8.2,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,http://image.yes24.com/goods/35552641/L,추리/미스터리
4,5,루팡의 딸,요코제키 다이,북플라자,2019.09,13500,31,8.5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...,http://image.yes24.com/goods/78860364/L,추리/미스터리


In [11]:
#스토리의 키워드를 뽑기위해 명사와 형용사만 가져오자
story_words = df_books_data['story'].to_numpy()
for i, document in enumerate(story_words):
    okt = konlpy.tag.Okt()
    clean_words = []
    for word in okt.pos(document, stem = True): #어간 뽑자
        if word[1] in ['Noun', 'Adjective']: #명사, 형용사만 가져와
            clean_words.append(word[0])
    #print(clean_words)
    document = ' '.join(clean_words)
    #print(document)
    story_words[i] = document

In [26]:
#넘파이 배열 데이터프레임 화
df_story_words=pd.DataFrame(story_words,
            columns = ['story_words'])

In [27]:
df_books = pd.concat([df_books_data, df_story_words], axis = 1)

In [40]:
df_books.columns

Index(['title', 'author', 'pub', 'price', 'review', 'rating', 'story', 'img',
       'genre', 'story_words'],
      dtype='object')

In [38]:
#날짜, 아이디 날리고
df_books = df_books.drop(['id','date'],axis = 1)


In [43]:
#혹시모를 중복 제거하자
df_books = df_books.drop_duplicates(['title'])

In [44]:
df_books.tail()

,title,author,pub,price,review,rating,story,img,genre,story_words
79768,그녀를 위해,,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,http://image.yes24.com/goods/373089/L,로맨스,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...
79769,하루만 사랑해 1,,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375482/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79770,하루만 사랑해 2,,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375484/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
79771,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스,
79772,kiss me...,,현무,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스,


In [56]:
#세트책들 날리자
df_books_tu=df_books[df_books['title'].str.contains('세트')==False]

In [58]:
df_books2 = df_books_tu.reset_index()

In [59]:
df_books2.tail()

,index,title,author,pub,price,review,rating,story,img,genre,story_words
73503,79768,그녀를 위해,,신영미디어,2880,0,0.0,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...,http://image.yes24.com/goods/373089/L,로맨스,마르코 고아 조카 안젤리나 사람 필요하다 양육 위 편의 상의 아내 성실하다 착하다 ...
73504,79769,하루만 사랑해 1,,파피루스(디앤씨미디어),8100,1,7.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375482/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
73505,79770,하루만 사랑해 2,,파피루스(디앤씨미디어),8100,0,0.0,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...,http://image.yes24.com/goods/375484/L,로맨스,날 서원 횡단보도 교통사고 당하다 상대 인천 상고 짱 노랗다 머 민 하루 놈 오히려...
73506,79771,서울 여자와 대구 남자가 만날 때,정민,영상노트,8100,0,0.0,,http://image.yes24.com/goods/376458/L,로맨스,
73507,79772,kiss me...,,현무,7200,2,5.0,,http://image.yes24.com/goods/377351/L,로맨스,


In [60]:
#인덱스 칼럼 날려줍시다
df_book = df_books2.drop(['index'], axis =1)

In [63]:
#아 스토리랑 스토리 워드랑 똑같아 졌다.. 둘다 갖고있지 말고 하나 날리자
df_book = df_book.drop(['story_words'],axis = 1)

In [67]:
df_book.head()

,title,author,pub,price,review,rating,story,img,genre
0,요리코를 위해,노리즈키 린타로,모모,13500,43,9.2,스스로 악마 이자 신 딸 살해 남자 심판 아버지 비극 출발 이자 마지막 끝내 실체 ...,http://image.yes24.com/goods/89392792/L,추리/미스터리
1,그녀는 다 계획이 있다,히가시노 게이고,하빌리스,14220,13,9.4,하나 보석 점 살인 사건 과거 미래 사건 경쾌하다 리듬 감 있다 터치 게이고 복고 ...,http://image.yes24.com/goods/97561319/L,추리/미스터리
2,블랙 쇼맨과 이름 없는 마을의 살인,히가시노 게이고,알에이치코리아(RHK),16200,54,9.1,아버지 살해 당하다 그것 결혼 소식 직후 행복하다 순간 비극 시작 대기업 취직 후 ...,http://image.yes24.com/goods/95712763/L,추리/미스터리
3,돌이킬 수 없는 약속,야쿠마루 가쿠,북플라자,13500,217,8.2,제 에드가 란 포상 수상 작가 야쿠 마루 가쿠 미스터리 추리 소설 내 딸 살해 놈 ...,http://image.yes24.com/goods/35552641/L,추리/미스터리
4,루팡의 딸,요코제키 다이,북플라자,13500,31,8.5,도둑 집안 대부 변 사체 발견 도둑 집안 대부 이 오 하나 코 손녀 딸 있다 하나 ...,http://image.yes24.com/goods/78860364/L,추리/미스터리


In [71]:
#데이터 엑셀로 편하게 뽑아놓자
df_book.to_excel('book_preprocessing.xlsx')

C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/sysimage/pd_19_L.gif' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/11274203/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/11267963/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/11263128/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % 

C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/2495535/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/2495533/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/2495532/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/2495551/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_un

C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/1945060/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/1945631/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/1945633/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_unicode(url))
C:\Users\82102\Anaconda3\lib\site-packages\xlsxwriter\worksheet.py:951: UserWarning: Ignoring URL 'http://image.yes24.com/goods/1946108/L' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  "65,530 URLS per worksheet." % force_un

In [2]:
import pip
!pip freeze

absl-py @ file:///tmp/build/80754af9/absl-py_1607439979954/work
alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.7
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1610472525955/work
appdirs==1.4.4
argh==0.26.2
argon2-cffi @ file:///C:/ci/argon2-cffi_1613038019788/work
asgiref==3.2.10
asn1crypto @ file:///tmp/build/80754af9/asn1crypto_1596577642040/work
astor==0.8.1
astroid @ file:///C:/ci/astroid_1592481955828/work
astropy @ file:///C:/ci/astropy_1606919052914/work
async-generator==1.10
atomicwrites==1.4.0
attrs @ file:///tmp/build/80754af9/attrs_1604765588209/work
autopep8 @ file:///tmp/build/80754af9/autopep8_1613246362269/work
Babel @ file:///tmp/build/80754af9/babel_1607110387436/work
backcall @ file:///home/ktietz/src/ci/backcall_1611930011877/work
backports.shutil-get-terminal-size @ file:///tmp/build/80754af9/backports.shutil_get_terminal_size_1608222128777/work
backtrader==1.9.76.123
bcrypt @ file:///C:/ci/bcrypt_1597918112552/work
beautifulsoup4=


ipykernel @ file:///C:/ci/ipykernel_1596208728219/work/dist/ipykernel-5.3.4-py3-none-any.whl
ipython @ file:///C:/ci/ipython_1612213113434/work
ipython-genutils @ file:///tmp/build/80754af9/ipython_genutils_1606773439826/work
ipywidgets @ file:///tmp/build/80754af9/ipywidgets_1610481889018/work
isort @ file:///tmp/build/80754af9/isort_1612243941122/work
itsdangerous==1.1.0
jdcal==1.4.1
jedi @ file:///C:/ci/jedi_1606932641827/work
Jinja2 @ file:///tmp/build/80754af9/jinja2_1612213139570/work
joblib @ file:///tmp/build/80754af9/joblib_1607970656719/work
JPype1 @ file:///C:/Users/82102/jupi%20python/Machinelearning/%EB%8F%84%EC%84%9C%20%EC%B6%94%EC%B2%9C%20%EC%8B%9C%EC%8A%A4%ED%85%9C%20%EA%B5%AC%ED%98%84/JPype1-1.2.0-cp37-cp37m-win_amd64.whl
json5==0.9.5
jsonschema @ file:///tmp/build/80754af9/jsonschema_1602607155483/work
jupyter==1.0.0
jupyter-client @ file:///tmp/build/80754af9/jupyter_client_1601311786391/work
jupyter-console @ file:///tmp/build/80754af9/jupyter_console_1598884538475